In [ ]:
# 改用hdbscan取代

In [1]:
import hdbscan
import pandas as pd,os,sqlite3,warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
warnings.filterwarnings('ignore', category=UserWarning)
os.chdir(r'C:\Users\018363\PythonCode')
import SenaoDB
from math import ceil
from datetime import datetime
import time
import numpy as np
# pd.Timestamp.today().strftime('%Y%m%d')

In [ ]:
conn=sqlite3.connect(r'C:\Users\018363\Project\20230617_sqlite3\Lab.db')
sql="""
select * from dataprocessing
"""
df=pd.read_sql(sql,conn)
df.shape

In [ ]:
df_clean=df.iloc[:,1:].apply(lambda x:x.fillna(x.median()),axis=0)

In [ ]:
# hdb=hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
#     gen_min_span_tree=False, leaf_size=40,core_dist_n_jobs=-1,
#     metric='euclidean', min_cluster_size=1000, min_samples=2000, p=None
#                     # , prediction_data=True                   
#                    )

In [ ]:
# %%time
# hdb.fit(df_clean.iloc[:,1:])
# pd.Series(hdb.labels_).value_counts()

In [ ]:
%%time
#訓練分類器
hdb_model=hdbscan.HDBSCAN(min_cluster_size=350, min_samples=500, prediction_data=True,core_dist_n_jobs=-1).fit(df_clean.iloc[:,1:])
pd.Series(hdb_model.labels_).value_counts()

In [ ]:
#寫入sqlite
def Insert_TableV2(df):
    conn=sqlite3.connect(r'C:\Users\018363\Project\20230617_sqlite3\Lab.db')     
    df.to_sql("hdbscan_label_"+pd.Timestamp.today().strftime('%Y%m%d'), conn, if_exists='append', index=False)    
    print('寫入Lab.db') 
    return()

In [ ]:
#設計隱函數1，T/F Y/N 轉換1/0 TF_list存放有被轉換的欄位
def TF_encode(data, col):
    data[col] = [1 if  i=='Y'  else 0 for i in data[col] ]
    return data

#設計隱函數2，提取日期欄位的年分
def year_encode(data, col):
    data[col + '_year']=[i.year if i!=0 else 0 for i in data[col]]
    return data

def trans_format(df):
    global tp
    tp=df.copy()
    for i in tp.select_dtypes(include=['object']):
        tp[i]=tp[i].astype('category')
       
    # #原始欄位
    raw_columns=df.columns
    
    #日期欄位-需轉換
    date_columns=tp.select_dtypes(include=['datetime64[ns]']).columns.to_list()
    #二元非1/0欄位-需轉換
    binary_colums=[i for i in tp.columns if  i.endswith('click')|i.endswith('pushing')|i.endswith('open')
               |i.endswith('login')|i.startswith('is_')|i.endswith('flag')|i.startswith('active')]
    #未轉換過的欄位(不曾轉換過&轉換完的欄位)
    keeping=[x for x in raw_columns if x not in binary_colums+date_columns]
    
    for _ in binary_colums:
        TF_encode(tp,_)
    print('完成二元欄位轉換')
    
    for _ in date_columns:
        year_encode(tp,_)
    print('完成年度欄位轉換')
    
    #月日改為差距天數
    for col in tp.select_dtypes(include=['datetime64[ns]']):
        tp[col]=[(datetime.strptime(str(i)[:10], '%Y-%m-%d')-datetime.strptime('1900-01-01', '%Y-%m-%d')).days if pd.notna(i) else 0 for i in tp[col] ] 
    print('完成天數轉換')  
    
    return()

# 讀取全會員並且批次分類

In [2]:
db=SenaoDB.DB()

In [ ]:
# 每個塊的大小
batch_size = 100000

# 獲取數據集的總大小
sql="""
SELECT COUNT(*) from pmart.member_tag_mart  
--where first_ec_order_date is not NULL and active_member='Y'
"""
db.cursor.execute(sql)
total_records = db.cursor.fetchone()[0]
print('全部筆數：',total_records)

# 計算塊數量
# num_batches = total_records // batch_size
num_batches = ceil(total_records / batch_size)
print('迭代次數：',num_batches)

In [ ]:
# %%time
#迭代訓練
for batch_idx in range(num_batches):
# for batch_idx  in range(0,1):
# for batch_idx  in range(297,num_batches):
    start_time = time.time()  # 開始時間
    # 查詢塊內的數據
    db.cursor.execute("""SELECT
                 member_id
                ,age
                ,birthday ::DATE
                ,sex
                ,active_member
                ,app_latest_used_date
                ,auth_date
                ,is_accept_email
                ,is_accept_sms
                ,is_email_verified
                ,cht_final_bis_date
                ,cht_pre_amt
                ,cht_pre_period
                ,cht_tenure
                ,is_txn
                ,distributor_rfm_flag
                ,first_ec_order_date
                ,first_o2o_order_date
                ,first_order_date
                ,first_pos_order_date
                ,first_pos_order_date_contract
                ,lastest_ec_order_date
                ,lastest_o2o_order_date
                ,lastest_order_date
                ,lastest_pos_order_date
                ,lastest_pos_order_date_contract
                ,click_1month_before_pushing
                ,click_2months_before_pushing
                ,click_3months_before_pushing
                ,click_1month_after_pushing
                ,click_2months_after_pushing
                ,click_3months_after_pushing
                ,this_month_click
                ,this_month_open
                ,last_month_open
                ,this_month_open_click
                ,last_month_open_click
                ,last_3months_login
                from pmart.member_tag_mart OFFSET %s LIMIT %s
                """, (batch_idx * batch_size, batch_size))
    rows = db.cursor.fetchall()
    
    # 獲取欄位名稱
    columns = [desc[0] for desc in db.cursor.description]

    # 將結果轉換為DataFrame
    df = pd.DataFrame(rows, columns=columns).fillna(0)
    
    #原始欄位
    raw_columns=df.columns
#     print('處理前欄位列表',raw_columns)
    #日期欄位-需轉換
    date_columns=[i for i in df.columns if  i.endswith('date') | i.endswith('_date_contract')|i.startswith('birth') | i.endswith('day')]

    #二元非1/0欄位-需轉換
    binary_colums=[i for i in df.columns if  i.endswith('click')|i.endswith('pushing')|i.endswith('open')
               |i.endswith('login')|i.startswith('is_')|i.endswith('flag')|i.startswith('active')]

    #未轉換過的欄位(不曾轉換過&轉換完的欄位)
    keeping=[x for x in raw_columns if x not in binary_colums+date_columns]  
    
    #轉時間
    for i in date_columns:
        try:
            df[i]=pd.to_datetime(df[i], errors='coerce',format='%Y-%m-%d  %H:%M:%S')    
#     tp[date_columns] = pd.to_datetime(tp[date_columns], errors='coerce',format='%Y-%m-%d')
        except:
            print(i)
    
    #轉文字
    #for i in tp.select_dtypes(include=['category']).columns.tolist():
    #    tp[i]=tp[i].astype('object')
        
    df['cht_pre_period']=df['cht_pre_period'].astype(int)
    
    df['cht_pre_amt']=df['cht_pre_amt'].astype(int)
    
   #執行轉換
    trans_format(df)    
            
    #one-hot-encoding
    one_hot_columns=[]
    one_hot_columns=tp.select_dtypes(include=['category']).columns.tolist()
    one_hot_columns.remove('member_id')
    
    #clone df
    df3=tp[one_hot_columns].copy()
    df_dummy=pd.get_dummies(df3,dtype=int)    
    df_final=pd.concat([tp.loc[:,~tp.columns.isin(one_hot_columns)], df_dummy], axis=1)
#     df_final=tp.loc[:,~tp.columns.isin(one_hot_columns)]
    print('分類資料維度：',df_final.shape)
    # 將數據轉換為NumPy數組
    # X = np.array(df_final.iloc[:,1:].fillna(0))

    #可以不轉nparray
    X = df_final.iloc[:,1:].fillna(0)

    # 使用分類器分類
    test_labels, strengths = hdbscan.approximate_predict(hdb_model,X.iloc[:,1:])
    end_time = time.time()  # 結束時間
    processing_time = end_time - start_time  # 處理時間（秒）
    print("Processed batch", batch_idx + 1, "of", num_batches,'處理時間(秒)：',processing_time)

#獲取每個樣本的分類
    labels = test_labels

#分群結果
    result=pd.DataFrame({'member_id':df['member_id'],'Group':labels,'CreateDate':datetime.today().strftime('%Y%m%d')})
    print(result['Group'].value_counts().sort_index())

#將結果存儲到SQLite3
    try:
        Insert_TableV2(result)
    except psycopg2.OperationalError as e:
        print("資料庫連結異常：", str(e))

try:
    engine.dispose()
except:
    pass
# 提交並關閉連接
# conn.commit()
# cur.close()
# conn.close()

In [ ]:
# df_dummy

In [ ]:
#分類新數據
# test_labels, strengths = hdbscan.approximate_predict(hdb_model,df_clean.iloc[10000:10100,1:])
# test_labels

In [ ]:
# hdbscan_result=df_clean.iloc[:40000,:1].copy()
# hdbscan_result=df.iloc[:,:1].copy()
# hdbscan_result['HDBSCAN']=hdb.labels_
# hdbscan_result['createdate']=pd.Timestamp.today().strftime('%Y%m%d')

# 讀取結果

In [ ]:
#%%time
#檢查結果
conn=sqlite3.connect(r'C:\Users\018363\Project\20230617_sqlite3\Lab.db')
sql=f"""
select * from {"hdbscan_label_"+datetime.today().strftime('%Y%m%d')}
"""
df22=pd.read_sql(sql,conn)
df22.shape

In [ ]:
#寫入EDW
def Insert_Table(df,table_name,schema):
    db=SenaoDB.DB()
    engine=db._engine
    df.to_sql(table_name, engine,schema=schema,if_exists='replace',index=False)    
    print('寫入')
    return()
    engine.dispose()
    db.close()

In [ ]:
Insert_Table(df22,'anio_hdbscan_result','ptest')

# 計算各群推薦賣場

In [3]:
help(db)

Help on DB in module SenaoDB object:

class DB(builtins.object)
 |  這個套件是用來連線神腦EDW
 |  使用套件，初始化DB()會直接連接DB、也會啟動db._conn
 |  db=DB()    
 |  可以使用的函數是：
 |  1.db._connect()    
 |  2.db.close()    
 |  可以使用的語法是：
 |  db.cursor.execute(sql)
 |  db.connection.commit()  
 |  
 |  可以使用的物件
 |  cnxn=db._conn
 |  engine=db._engine
 |  
 |  上傳資料範例
 |  df.to_sql('anio_StoreTrafficV6', engine, schema='ptest', if_exists='append', index=False)
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  close(self)
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  connection
 |  
 |  cursor
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

In [ ]:
sql="""
--step1.執行迴圈
truncate table ptest.Group_recommend_result;

select ptest.anio_recommend_by_cluster_hdbscan();
"""
db.cursor.execute(sql)
db.connection.commit()